In [1]:
import tensorflow as tf

import numpy as np

2023-04-15 13:24:43.632600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 13:24:44.567347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-15 13:24:48.842041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-15 13:24:48.895513: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-15 13:24:48.895562: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
gpu = tf.device('/GPU:0')
with gpu:
    # Create two tensors and perform element-wise multiplication
    a = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
    b = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)
    c = tf.multiply(a, b)
    print(c)


tf.Tensor(
[[ 5. 12.]
 [21. 32.]], shape=(2, 2), dtype=float32)


In [6]:
with gpu:
    # Create two tensors and perform element-wise multiplication
    a = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
    b = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)
    c = tf.multiply(a, b)
    print(c)

tf.Tensor(
[[ 5. 12.]
 [21. 32.]], shape=(2, 2), dtype=float32)


In [7]:
# import urllib.request
# import os

# mnist_dir = './mnist_data/'
# if not os.path.exists(mnist_dir):
#     os.makedirs(mnist_dir)

# # Download and save the MNIST data files
# base_url = 'http://yann.lecun.com/exdb/mnist/'
# files = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',
#          't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']
# for file in files:
#     url = base_url + file
#     filepath = mnist_dir + file
#     urllib.request.urlretrieve(url, filepath)
#     print(f'Downloaded {file} to {mnist_dir}')


In [8]:
# preprocess data

import gzip
import numpy as np

# Set the file paths for the MNIST digit dataset files
train_images_path = './mnist_data/train-images-idx3-ubyte.gz'
train_labels_path = './mnist_data/train-labels-idx1-ubyte.gz'
test_images_path = './mnist_data/t10k-images-idx3-ubyte.gz'
test_labels_path = './mnist_data/t10k-labels-idx1-ubyte.gz'


# Load the MNIST digit dataset files into numpy arrays
with gzip.open(train_images_path, 'rb') as f:
    x_train = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)
    
with gzip.open(train_labels_path, 'rb') as f:
    y_train = np.frombuffer(f.read(), np.uint8, offset=8)

with gzip.open(test_images_path, 'rb') as f:
    x_test = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28)
    
with gzip.open(test_labels_path, 'rb') as f:
    y_test = np.frombuffer(f.read(), np.uint8, offset=8)

    
    
# to make the value on each pixel in range (0,1)
x_train = x_train / 255    
x_test = x_test / 255    


# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

x_train.shape, x_test.shape

# change the type to be ready for the training process
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)





2023-04-15 13:25:10.693016: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-04-15 13:25:10.781233: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


In [9]:
# model
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D

# with gpu:
class LowLeNet(Model):
    def __init__(self, input_shape=(32, 32, 1), nb_classes=10):
        super().__init__()
        self.create_model(input_shape = input_shape, nb_classes = nb_classes)
        # initialize some useful properties to be able to emulate the behavior of a distributed sytem on a local machine
        self.global_weights_path = './result/global'
        self.parent_upload_folder = './result/local'
        # weights related
        self.global_weights = []
        self.previous_weights = []
        self.current_global_version = 1
        self.current_global_path = ''
        # info
        self.dataset_size = 10000
        self.worker_name = 'worker1'
        self.result = []
        self.direction = []

        
    def create_model(self, input_shape, nb_classes):
        self.conv1 = Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="valid")
        self.avgpool1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')
        self.avgpool2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='tanh')
        self.dense2 = Dense(84, activation='tanh')
        self.dense3 = Dense(nb_classes, activation='softmax')
        

    def call(self, x):
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        x = self.avgpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    def compile(self, loss = "categorical_crossentropy", optimizer = "Adam", metric = "accuracy"):
        # Now, for this model, support only this set of choice 
        if loss == "categorical_crossentropy":
        # define loss = Categorical Crossentropy
            self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        if optimizer == "Adam":
        # define optimizer = Adam
            self.optimizer = tf.keras.optimizers.Adam()
        if metric == "accuracy":
            # setting up metric = accuracy
            self.train_loss = tf.keras.metrics.Mean(name='train_loss')
            self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
            self.test_loss = tf.keras.metrics.Mean(name='test_loss')
            self.test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    @tf.function
    def test_step(self, images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = self(images, training=False)
        t_loss = self.loss_object(labels, predictions)
        self.test_loss(t_loss)
        self.test_accuracy(labels, predictions)
        
    @tf.function
    def train_step(self, images, labels):
        with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
            predictions = self(images, training=True)
            loss = self.loss_object(labels, predictions)
        # compute the gradient based on the loss
        gradients = tape.gradient(loss, self.trainable_variables)
        # update the weights based on some formula determined by the chosen algorithm
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # update training info
        self.train_loss(loss)
        self.train_accuracy(labels, predictions)

    # Utils
    # --------------------------------------------------------------------------

    def load_pretrained_model(self, weights, train_ds):
        num_of_layers= len(self.get_weights())
        if num_of_layers < len(weights):
            for images, labels in train_ds:
                self.train_step(images, labels)
                break
        self.set_weights(weights)

    # file related
    def get_latest_global_version(self):
        path = self.global_weights_path
        num = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
        return num
    
    def get_latest_global_weights(self, version_num):
        keyword = f"version{version_num}"
        for file_name in os.listdir(self.global_weights_path):
            if keyword in file_name:
                file_path = os.path.join(self.global_weights_path, file_name)
                self.current_global_path = file_path
                break
        with open(file_path, 'rb') as f:
            weights = pickle.load(f)
        return weights 

    def upload_weights_to_file(self, local_epoch):
        # global_epoch_num = 0
        file_name = f"{self.worker_name}_{self.dataset_size}_{local_epoch}_{self.current_global_version}.pkl"
        path = self.parent_upload_folder
        global_upload_folder_num = len([f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))])
        folder_name = f"epoch{global_upload_folder_num}"
        upload_path = os.path.join(self.parent_upload_folder, folder_name, file_name)
        with open(upload_path, 'wb') as f:
            pickle.dump(self.get_weights(), f)
        return upload_path
    
    # --------------------------------------------------------------------------

    def merge(self):

        # updating the value in each parameters of the local model
        # calculate the different to decide the formula of updating
            # the calculation is performed on each corresponding layout
        # the different between the current weight and the previous weights
        if len(self.previous_weights) < len(self.current_weights):
            # applying when entering the training step with an initialized model (8 layers), not the pretrained one (18 layers)
            # in the 1st batch of the 1st epoch
            e_local = self.current_weights
        else:
            # perform calculation normally for all the other cases
            e_local = [layer_a - layer_b for layer_a, layer_b in zip(self.current_weights, self.previous_weights)]

        # the different between the global weights and the current weights
        e_global = [layer_a - layer_b for layer_a, layer_b in zip(self.global_weights, self.current_weights)]

        # check the dimension of these variables to see whether it fits one another
        print(f"total layers of previous weights: {len(self.previous_weights)}, total layers of current weights: {len(self.current_weights)}")
        print(f"total layers of e_local: {len(e_local)}, total layers of e_global: {len(e_global)}")
            # get the direction list (matrix)
        self.direction = [np.multiply(a, b) for a, b in zip(e_local, e_global)]
        print(f"total layers of direction: {len(self.direction)}")

        # calculate alpha variable to ready for the merging process
        current_global_size = int(self.current_global_path.split('/')[-1].split('.')[0].split('_')[-1])
        alpha = (self.dataset_size) / (self.dataset_size + current_global_size)

        # create a blank array to store the result
        self.result = [np.zeros(layer.shape) for layer in self.current_weights]
            # base on the direction, global weights and current local weights
            # updating the value of each parameter to get the new local weights (from merging process)
        # set the index to move to the next layer
        t = 0
        # check some param of the merging process
        self.total = 0
        self.different_direction = 0
        self.same_direction = 0
        # access each layer of these variables correspondingly 
        for (local_layer, global_layer, direction_layer) in zip(self.current_weights, self.global_weights, self.direction):
            # print(local_layer.shape, global_layer.shape, direction_layer.shape)
            # access each element in each layer
            it = np.nditer([local_layer, global_layer, direction_layer], flags=['multi_index'])
            for local_element, global_element, direction_element in it:
                self.total += 1
                index = it.multi_index

                if direction_element >= 0:
                    result_element = global_element
                    self.different_direction += 1
                else:
                    result_element = (1 - alpha)*global_element + alpha * local_element
                    self.same_direction += 1

                self.result[t][index] = result_element
            # move to the next layer
            t +=1
            # set the current weights of the model to be the result of the merging process
        self.set_weights(self.result)
        
    # -------------------------------oOo-------------------------------------------
    def train(self, train_ds, test_ds, EPOCHS = 5,):
        # Training for multiple epoch
        for epoch in range(EPOCHS):
            # Reset the metrics at the start of the next epoch
            self.train_loss.reset_states()
            self.train_accuracy.reset_states()
            self.test_loss.reset_states()
            self.test_accuracy.reset_states()
            # self.second_weight = self.get_weights()

            batch_num = 0
            for images, labels in train_ds:
                batch_num +=1
                # record the local weights before it enter a new training batch
                self.previous_weights = self.get_weights()
                # begin the local training process of each batch
                self.train_step(images, labels)
                # check whether there is a new global model --> base on the number of files in the folder result/global/
                latest_global_version = self.get_latest_global_version()
                # if there is a new global version --> ready to merge with the local training result
                if self.current_global_version < latest_global_version:
                    print(f"Merging process happens at epoch {epoch}, batch {batch_num} when receiving the global version {latest_global_version}, current global version {self.current_global_version}")
                    # update the current global version on local machine
                    self.current_global_version = latest_global_version
                    # load the new global weights 
                    self.global_weights = self.get_latest_global_weights(self.current_global_version)
                    # get the current local weights --> the weights obtained after executing the self.train_step() function
                    self.current_weights = self.get_weights()
                    # check whether the number of layers of previous and current weights match
                    if len(self.previous_weights) < len(self.current_weights):
                        print(f"The number of layers of previous weights: {len(self.previous_weights)}")
                        print(f"The number of layers of current weights: {len(self.current_weights)}")
                        print(f"The previous weights that mismatch in the number of layers with the current weights {self.previous_weights}")
                    # check whether the number of layers of the current weights and the global weights match
                    if len(self.global_weights) != len(self.current_weights):
                        print(f"THERE IS A MISMATCH IN NUMBER OF LAYERS OF THE GLOBAL WEIGHTS AND CURRENT WEIGHTS")
                        print(f"EXIT THE MERGING PROCESS")
                        print(f"The number of layers of global weights: {len(self.global_weights)}")
                        print(f"The number of layers of current weights: {len(self.current_weights)}")
                        break
                    else:
                    # merged based on previous local weights, current local weights and global weights
                        self.merge()
                        self.merged_weights = self.get_weights()

                # else:
                # if there is no new global model --> update the local weights normally 
                # --> no modification to the local weights
                # do nothing
    # -------------------------------oOo-------------------------------------------

            for test_images, test_labels in test_ds:
                self.test_step(test_images, test_labels)

            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {self.train_loss.result()}, '
                f'Accuracy: {self.train_accuracy.result() * 100}, '
                f'Test Loss: {self.test_loss.result()}, '
                f'Test Accuracy: {self.test_accuracy.result() * 100}'
                )

            # updating local weight to appropriate directory
            latest_global_version = self.get_latest_global_version()
            upload_path = self.upload_weights_to_file(epoch)
            print(f"\tFrom {self.worker_name} in local epoch {epoch + 1} that hold the global version {self.current_global_version}, uploading local model to the server in the path {upload_path}")

# #  create and compile model
#     model= LowLeNet()
#     model.compile()

#     model.train(train_ds, test_ds, EPOCHS = 5)

#     current_weights = model.get_weights() 
#     for layer in current_weights:
#         print(layer.shape)



    

In [ ]:
model= LowLeNet()
model.compile()

model.train(train_ds, test_ds, EPOCHS = 5)

current_weights = model.get_weights() 
for layer in current_weights:
    print(layer.shape)

2023-04-15 13:25:17.764628: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2023-04-15 13:25:17.910489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-04-15 13:25:17.910816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
/home/vtn_ubuntu/miniconda3/envs/fedasync/lib/python3.9/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, bu

In [9]:
with gpu:
#  create and compile model
    model= LowLeNet()
    model.compile()

    model.train(train_ds, test_ds, EPOCHS = 5)

    current_weights = model.get_weights() 
    for layer in current_weights:
        print(layer.shape)



2023-04-15 12:54:46.442817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,28,28,1]
	 [[{{node Placeholder/_0}}]]
2023-04-15 12:54:46.443169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
/home/vtn_ubuntu/miniconda3/envs/fedasync/lib/python3.9/site-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_l

: 

: 

In [ ]:
# model
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D

with gpu:
    class LowLeNet(Model):
        def __init__(self, input_shape=(32, 32, 1), nb_classes=10):
            super().__init__()
            self.create_model(input_shape = input_shape, nb_classes = nb_classes)
            # initialize some useful properties to be able to emulate the behavior of a distributed sytem on a local machine
            self.global_weights_path = './result/global'
            self.parent_upload_folder = './result/local'
            # weights related
            self.global_weights = []
            self.previous_weights = []
            self.current_global_version = 1
            self.current_global_path = ''
            # info
            self.dataset_size = 10000
            self.worker_name = 'worker1'
            self.result = []
            self.direction = []

            
        def create_model(self, input_shape, nb_classes):
            self.conv1 = Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape, padding="valid")
            self.avgpool1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
            self.conv2 = Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')
            self.avgpool2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')
            self.flatten = Flatten()
            self.dense1 = Dense(120, activation='tanh')
            self.dense2 = Dense(84, activation='tanh')
            self.dense3 = Dense(nb_classes, activation='softmax')
            

        def call(self, x):
            x = self.conv1(x)
            x = self.avgpool1(x)
            x = self.conv2(x)
            x = self.avgpool2(x)
            x = self.flatten(x)
            x = self.dense1(x)
            x = self.dense2(x)
            x = self.dense3(x)
            return x
        
        def compile(self, loss = "categorical_crossentropy", optimizer = "Adam", metric = "accuracy"):
            # Now, for this model, support only this set of choice 
            if loss == "categorical_crossentropy":
            # define loss = Categorical Crossentropy
                self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            if optimizer == "Adam":
            # define optimizer = Adam
                self.optimizer = tf.keras.optimizers.Adam()
            if metric == "accuracy":
                # setting up metric = accuracy
                self.train_loss = tf.keras.metrics.Mean(name='train_loss')
                self.train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
                self.test_loss = tf.keras.metrics.Mean(name='test_loss')
                self.test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

        @tf.function
        def test_step(self, images, labels):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
            predictions = self(images, training=False)
            t_loss = self.loss_object(labels, predictions)
            self.test_loss(t_loss)
            self.test_accuracy(labels, predictions)
            
        @tf.function
        def train_step(self, images, labels):
            with tf.GradientTape() as tape:
            # training=True is only needed if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
                predictions = self(images, training=True)
                loss = self.loss_object(labels, predictions)
            # compute the gradient based on the loss
            gradients = tape.gradient(loss, self.trainable_variables)
            # update the weights based on some formula determined by the chosen algorithm
            self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
            # update training info
            self.train_loss(loss)
            self.train_accuracy(labels, predictions)

        # Utils
        # --------------------------------------------------------------------------

        def load_pretrained_model(self, weights, train_ds):
            num_of_layers= len(self.get_weights())
            if num_of_layers < len(weights):
                for images, labels in train_ds:
                    self.train_step(images, labels)
                    break
            self.set_weights(weights)

        # file related
        def get_latest_global_version(self):
            path = self.global_weights_path
            num = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
            return num
        
        def get_latest_global_weights(self, version_num):
            keyword = f"version{version_num}"
            for file_name in os.listdir(self.global_weights_path):
                if keyword in file_name:
                    file_path = os.path.join(self.global_weights_path, file_name)
                    self.current_global_path = file_path
                    break
            with open(file_path, 'rb') as f:
                weights = pickle.load(f)
            return weights 

        def upload_weights_to_file(self, local_epoch):
            # global_epoch_num = 0
            file_name = f"{self.worker_name}_{self.dataset_size}_{local_epoch}_{self.current_global_version}.pkl"
            path = self.parent_upload_folder
            global_upload_folder_num = len([f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))])
            folder_name = f"epoch{global_upload_folder_num}"
            upload_path = os.path.join(self.parent_upload_folder, folder_name, file_name)
            with open(upload_path, 'wb') as f:
                pickle.dump(self.get_weights(), f)
            return upload_path
        
        # --------------------------------------------------------------------------

        def merge(self):

            # updating the value in each parameters of the local model
            # calculate the different to decide the formula of updating
                # the calculation is performed on each corresponding layout
            # the different between the current weight and the previous weights
            if len(self.previous_weights) < len(self.current_weights):
                # applying when entering the training step with an initialized model (8 layers), not the pretrained one (18 layers)
                # in the 1st batch of the 1st epoch
                e_local = self.current_weights
            else:
                # perform calculation normally for all the other cases
                e_local = [layer_a - layer_b for layer_a, layer_b in zip(self.current_weights, self.previous_weights)]

            # the different between the global weights and the current weights
            e_global = [layer_a - layer_b for layer_a, layer_b in zip(self.global_weights, self.current_weights)]

            # check the dimension of these variables to see whether it fits one another
            print(f"total layers of previous weights: {len(self.previous_weights)}, total layers of current weights: {len(self.current_weights)}")
            print(f"total layers of e_local: {len(e_local)}, total layers of e_global: {len(e_global)}")
                # get the direction list (matrix)
            self.direction = [np.multiply(a, b) for a, b in zip(e_local, e_global)]
            print(f"total layers of direction: {len(self.direction)}")

            # calculate alpha variable to ready for the merging process
            current_global_size = int(self.current_global_path.split('/')[-1].split('.')[0].split('_')[-1])
            alpha = (self.dataset_size) / (self.dataset_size + current_global_size)

            # create a blank array to store the result
            self.result = [np.zeros(layer.shape) for layer in self.current_weights]
                # base on the direction, global weights and current local weights
                # updating the value of each parameter to get the new local weights (from merging process)
            # set the index to move to the next layer
            t = 0
            # check some param of the merging process
            self.total = 0
            self.different_direction = 0
            self.same_direction = 0
            # access each layer of these variables correspondingly 
            for (local_layer, global_layer, direction_layer) in zip(self.current_weights, self.global_weights, self.direction):
                # print(local_layer.shape, global_layer.shape, direction_layer.shape)
                # access each element in each layer
                it = np.nditer([local_layer, global_layer, direction_layer], flags=['multi_index'])
                for local_element, global_element, direction_element in it:
                    self.total += 1
                    index = it.multi_index

                    if direction_element >= 0:
                        result_element = global_element
                        self.different_direction += 1
                    else:
                        result_element = (1 - alpha)*global_element + alpha * local_element
                        self.same_direction += 1

                    self.result[t][index] = result_element
                # move to the next layer
                t +=1
                # set the current weights of the model to be the result of the merging process
            self.set_weights(self.result)
            
        # -------------------------------oOo-------------------------------------------
        def train(self, train_ds, test_ds, EPOCHS = 5,):
            # Training for multiple epoch
            for epoch in range(EPOCHS):
                # Reset the metrics at the start of the next epoch
                self.train_loss.reset_states()
                self.train_accuracy.reset_states()
                self.test_loss.reset_states()
                self.test_accuracy.reset_states()
                # self.second_weight = self.get_weights()

                batch_num = 0
                for images, labels in train_ds:
                    batch_num +=1
                    # record the local weights before it enter a new training batch
                    self.previous_weights = self.get_weights()
                    # begin the local training process of each batch
                    self.train_step(images, labels)
                    # check whether there is a new global model --> base on the number of files in the folder result/global/
                    latest_global_version = self.get_latest_global_version()
                    # if there is a new global version --> ready to merge with the local training result
                    if self.current_global_version < latest_global_version:
                        print(f"Merging process happens at epoch {epoch}, batch {batch_num} when receiving the global version {latest_global_version}, current global version {self.current_global_version}")
                        # update the current global version on local machine
                        self.current_global_version = latest_global_version
                        # load the new global weights 
                        self.global_weights = self.get_latest_global_weights(self.current_global_version)
                        # get the current local weights --> the weights obtained after executing the self.train_step() function
                        self.current_weights = self.get_weights()
                        # check whether the number of layers of previous and current weights match
                        if len(self.previous_weights) < len(self.current_weights):
                            print(f"The number of layers of previous weights: {len(self.previous_weights)}")
                            print(f"The number of layers of current weights: {len(self.current_weights)}")
                            print(f"The previous weights that mismatch in the number of layers with the current weights {self.previous_weights}")
                        # check whether the number of layers of the current weights and the global weights match
                        if len(self.global_weights) != len(self.current_weights):
                            print(f"THERE IS A MISMATCH IN NUMBER OF LAYERS OF THE GLOBAL WEIGHTS AND CURRENT WEIGHTS")
                            print(f"EXIT THE MERGING PROCESS")
                            print(f"The number of layers of global weights: {len(self.global_weights)}")
                            print(f"The number of layers of current weights: {len(self.current_weights)}")
                            break
                        else:
                        # merged based on previous local weights, current local weights and global weights
                            self.merge()
                            self.merged_weights = self.get_weights()

                    # else:
                    # if there is no new global model --> update the local weights normally 
                    # --> no modification to the local weights
                    # do nothing
        # -------------------------------oOo-------------------------------------------

                for test_images, test_labels in test_ds:
                    self.test_step(test_images, test_labels)

                print(
                    f'Epoch {epoch + 1}, '
                    f'Loss: {self.train_loss.result()}, '
                    f'Accuracy: {self.train_accuracy.result() * 100}, '
                    f'Test Loss: {self.test_loss.result()}, '
                    f'Test Accuracy: {self.test_accuracy.result() * 100}'
                    )

                # updating local weight to appropriate directory
                latest_global_version = self.get_latest_global_version()
                upload_path = self.upload_weights_to_file(epoch)
                print(f"\tFrom {self.worker_name} in local epoch {epoch + 1} that hold the global version {self.current_global_version}, uploading local model to the server in the path {upload_path}")
 
# #  create and compile model
#     model= LowLeNet()
#     model.compile()

#     model.train(train_ds, test_ds, EPOCHS = 5)

#     current_weights = model.get_weights() 
#     for layer in current_weights:
#         print(layer.shape)



        